[![](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MartinSchobben/tuw_education_notebooks/15bcbc542b9c50306617af6f796bb6c8fd4fcf0d?urlpath=lab%2Ftree%2F01_notebook_as_an_app.ipynb)


![Image from [wikipedia](https://upload.wikimedia.org/wikipedia/commons/d/d4/Thomas_Bayes.gif)](https://upload.wikimedia.org/wikipedia/commons/d/d4/Thomas_Bayes.gif)


In [ ]:
%matplotlib widget

import numpy as np
from eomaps import Maps

from tuw_education_notebooks.view_bayes_flood import view_bayes_flood
from tuw_education_notebooks.calc_bayes_flood import sig0_dc, calc_likelihoods

## From Backscattering to Flood Mapping

This notebook explains how microwave ($\sigma^0$) backscattering (@fig-area) can be used to map the extent of a flood. We replicate in this exercise the work of @bauer-marschallinger_satellite-based_2022 on the TU Wien Bayesian-based flood mapping algorithm.

In the following lines we create a map with EOmaps [@quast_getting_2024] of the $\sigma^0$ backscattering values.


In [ ]:
#| label: fig-area
#| fig-cap: 'Area targeted for $\sigma^0$ backscattering is the Greek region of Thessaly, which experienced a major flood in February of 2018.'

m = Maps(ax=121, figsize=(12, 9), crs=Maps.CRS.Equi7_EU)
m.set_data(data=sig0_dc, x="x", y="y", parameter="SIG0", crs=Maps.CRS.Equi7_EU)
m.plot_map()
m.add_colorbar(label="$\sigma^0$ (dB)", pos=(0.1, 0.02, 0.8, 0.1))
m.add_scalebar(n=5)
m2 = m.new_map(ax=122, crs=Maps.CRS.Equi7_EU)
m2.set_extent(m.get_extent())
m2.add_wms.OpenStreetMap.add_layer.default()
m.show()

## Microwave Backscattering over Land and Water

Reverend Bayes was concerned with two events, one "the hypothesis" occurring before the other "the evidence". If we know the cause it is easy to logically deduce the probability of an effect. But in this case we want to deduce the probability of a cause from an observed effect, also known as "reversed probability". In the case of flood mapping we have $\sigma^0$ backscatter observations over land (effect) and we want to deduce the probability of flooding (F) and non-flooding (NF). 

Phrased differently we want to know the probability of flooding given a pixel's $\sigma^0$:

$$P(F|\sigma^0)$$

and the probability of a pixel being not flooded given a certain $\sigma^0$:

$$P(NF|\sigma^0).$$

Bayes showed that these can be deduced from the observation that forward and reversed probability are equal, so that:

$$P(F|\sigma^0)P(\sigma^0) = P(\sigma^0|F)P(F)$$

and

$$P(NF|\sigma^0)P(\sigma^0) = P(\sigma^0|NF)P(NF).$$


The forward probability of $\sigma^0$ given flood ($P(\sigma^0|F)$) and $\sigma^0$ given non-flood ($P(\sigma^0|NF)$) can be extracted from past information on backscattering over land and water surfaces. As seen in the sketch below (@fig-sat), the characteristic of backscattering over land and water differ considerably.

![Schematic backscattering over land and water. Image from [Geological Survey Ireland](https://www.gsi.ie/images/images/SAR_mapping_land_water.jpg)](https://www.gsi.ie/images/images/SAR_mapping_land_water.jpg){#fig-sat}

## Likelihoods

These so-called likelihoods of $P(\sigma^0|F)$ and $P(\sigma^0|NF)$ can thus be calculated from past backscattering information. Without going into the details of how these likelihoods are calculated, you can **click** on a pixel of the map to plot the likelihoods of $\sigma^0$ being governed by land or water.


In [ ]:
#| label: fig-lik
#| fig-cap: 'Likelihoods for $\sigma^0$ being associated with land or water for 1 pixel in the Greek area of Thessaly. Likelihoods are calculated over a range of $\sigma^0$. The pixel''s observed $\sigma^0$ is given with a vertical line. Click on the map to re-calculate and update this figure for another pixel in the study area. Map created with EOmaps [@quast_getting_2024].'
view_bayes_flood(sig0_dc)

## Posteriors

Having calculated the likelihoods, we can now move on to calculate the probability of (non-)flooding given a pixel's $\sigma^0$. These so-called "posteriors" need on more piece of information as can be seen in the equation above. We need the probability whether a pixel is flooded $P(F)$ or non-flooded $P(NF)$. We can of course not estimate this information precisely, so our best estimate is that the chance of flooding versus non-flooding of a pixel is 50-50: a coin flip. This subjective prior believe is a fundament of Bayesian statistics and we use the likelihood to updated our believe in this particular hypothesis. We now can also calculate the probability of backscattering $P(\sigma^0)$, as the weighted average of the water and land likelihoods. The latter ensures that our updated evidence (or posteriors) ranges between 0 to 1.

The following code block show how we calculate the priors.


In [ ]:
def calc_posteriors(water_likelihood, land_likelihood):
    evidence = (water_likelihood * 0.5) + (land_likelihood * 0.5)
    return (water_likelihood * 0.5) / evidence,  (land_likelihood * 0.5) / evidence

We can plot these posteriors of the probability of flooding and non-flooding again and compare these to pixel's measured $\sigma^0$. **Click** on a pixel to calculate the posterior probability.


In [ ]:
#| label: fig-post
#| fig-cap: 'Posterior probabilities for $\sigma^0$ of 1 pixel being associated with land for water in the Greek area of Thessaly. Click on the map to re-calculate and update this figure for another pixel in the study area. Map created with EOmaps [@quast_getting_2024].'
view_bayes_flood(sig0_dc, calc_posteriors)

## Flood Classification

We are now ready to combine all this information and classify the pixels according to the probability of flooding given the backscaterr value of 1 pixel. Here we just look whether the probability of flooding is higher then non-flooding:


In [ ]:
def bayesian_flood_decision(id, sig0_dc):
    nf_post_prob, f_post_prob = calc_posteriors(*calc_likelihoods(id,  sig0_dc))
    return np.greater(f_post_prob, nf_post_prob)

**Click** on a point in the below map to see the likelihoods and posterior distributions (in the left-hand subplots).


In [ ]:
#| label: fig-clas
#| fig-cap: 'Flood extent of the Greek region of Thessaly based on Bayesian probabilities are shown on the map superimposed on an open street map. Click on a pixel to generate the point''s water and land likelihoods as well as the posterior probabilities. Map created with EOmaps [@quast_getting_2024].'
view_bayes_flood(sig0_dc, calc_posteriors, bayesian_flood_decision)